In [4]:
import torch
from torch_geometric.datasets import QM9
from torch_geometric.loader import DataLoader
from sklearn.model_selection import train_test_split
from torch.nn import Linear


dataset = QM9(root='./data/QM9')


torch.manual_seed(42) 
dataset = dataset.shuffle()

train_idx, temp_idx = train_test_split(range(len(dataset)), test_size=0.2, random_state=42)
val_idx, test_idx = train_test_split(temp_idx, test_size=0.5, random_state=42)

train_dataset = dataset[train_idx]
val_dataset = dataset[val_idx]
test_dataset = dataset[test_idx]

BATCH_SIZE = 64

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

TARGET_INDEX = 7
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_node_features = dataset.num_node_features
hidden_channels = 64
EPOCHS = 15 
criterion = torch.nn.L1Loss() 

print(f"num training data: {len(train_dataset)}")
print(f"num validation data: {len(val_dataset)}")
print(f"num test data: {len(test_dataset)}")

num training data: 104664
num validation data: 13083
num test data: 13084


In [5]:
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import os

LOG_DIR = 'runs'
os.makedirs(LOG_DIR, exist_ok=True)
CHECKPOINT_DIR = 'checkpoints'
os.makedirs(CHECKPOINT_DIR, exist_ok=True)


def evaluate(loader, model):
    model.eval()
    total_error = 0

    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            target = data.y[:, TARGET_INDEX].unsqueeze(1)
            out = model(data)
            
            loss = criterion(out, target) 
            total_error += loss.item() * data.num_graphs
    return total_error / len(loader.dataset)


def train_model(model, optimizer, name):
    print(f"\n===== train start: {name} =====")
    model.to(device)
    best_val_mae = float('inf')
    
    writer = SummaryWriter(os.path.join(LOG_DIR, name))
    
    checkpoint_path = os.path.join(CHECKPOINT_DIR, f'{name}_best.pt')
    
    for epoch in range(1, EPOCHS + 1):
        model.train()
        total_loss = 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            
            target = data.y[:, TARGET_INDEX].unsqueeze(1)
            out = model(data)
            
            loss = criterion(out, target)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * data.num_graphs
            
        train_mae = total_loss / len(train_loader.dataset)
        val_mae = evaluate(val_loader, model)
        
        writer.add_scalar('MAE/Train', train_mae, epoch)
        writer.add_scalar('MAE/Validation', val_mae, epoch)
        
        if val_mae < best_val_mae:
            best_val_mae = val_mae
            torch.save(model.state_dict(), checkpoint_path)
            print(f"Best model saved with Val MAE: {best_val_mae:.4f}")
            
        print(f'Epoch: {epoch:02d}/{EPOCHS}, Train MAE: {train_mae:.4f}, Val MAE: {val_mae:.4f} (Best: {best_val_mae:.4f})')
        
    writer.close()
    
    if os.path.exists(checkpoint_path):
        model.load_state_dict(torch.load(checkpoint_path))
    
    test_mae = evaluate(test_loader, model)
    print(f"=================================")
    print(f"final results {name} MAE: {test_mae:.4f}")
    return test_mae

In [6]:
from torch_geometric.nn import GINConv, global_add_pool

class GIN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super().__init__()

        self.conv1 = GINConv(Linear(in_channels, hidden_channels))
        self.conv2 = GINConv(Linear(hidden_channels, hidden_channels))
        self.conv3 = GINConv(Linear(hidden_channels, hidden_channels))
        
        self.lin1 = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, 1) 
        
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = F.relu(self.conv3(x, edge_index))

        x = global_add_pool(x, batch) 
        
        x = F.relu(self.lin1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        return self.lin2(x)

model_gin = GIN(num_node_features, hidden_channels)
optimizer_gin = torch.optim.Adam(model_gin.parameters(), lr=0.001)
train_model(model_gin, optimizer_gin, "GIN")


===== train start: GIN =====
Best model saved with Val MAE: 929.4878
Epoch: 01/15, Train MAE: 2337.7697, Val MAE: 929.4878 (Best: 929.4878)
Best model saved with Val MAE: 761.6856
Epoch: 02/15, Train MAE: 1893.9807, Val MAE: 761.6856 (Best: 761.6856)
Best model saved with Val MAE: 516.8287
Epoch: 03/15, Train MAE: 1810.6599, Val MAE: 516.8287 (Best: 516.8287)
Epoch: 04/15, Train MAE: 1781.9870, Val MAE: 613.8860 (Best: 516.8287)
Epoch: 05/15, Train MAE: 1760.7804, Val MAE: 815.3360 (Best: 516.8287)
Best model saved with Val MAE: 356.4003
Epoch: 06/15, Train MAE: 1750.9598, Val MAE: 356.4003 (Best: 356.4003)
Epoch: 07/15, Train MAE: 1756.9425, Val MAE: 364.8271 (Best: 356.4003)
Epoch: 08/15, Train MAE: 1733.9678, Val MAE: 612.0268 (Best: 356.4003)
Epoch: 09/15, Train MAE: 1724.8210, Val MAE: 801.6935 (Best: 356.4003)
Epoch: 10/15, Train MAE: 1727.6436, Val MAE: 425.7948 (Best: 356.4003)
Epoch: 11/15, Train MAE: 1723.1812, Val MAE: 474.6507 (Best: 356.4003)
Best model saved with Val MAE

/tmp/ipykernel_17915/2230799113.py:66: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(checkpoint_path))


final results GIN MAE: 109.0043


109.00427866754674

In [7]:
from torch_geometric.nn import PNAConv, global_add_pool
from torch_geometric.utils import degree

max_deg = -1
for data in dataset:
    max_deg = max(max_deg, int(degree(data.edge_index[0]).max().item()))

deg = torch.zeros(max_deg + 1, dtype=torch.long)
for data in dataset:
    d = degree(data.edge_index[0], dtype=torch.long)
    deg += torch.bincount(d, minlength=deg.numel())

mean_deg = (deg.view(-1).float() * torch.arange(deg.numel()).float()).sum() / deg.sum()
std_deg = torch.sqrt(((deg.view(-1).float() * (torch.arange(deg.numel()).float() - mean_deg) ** 2).sum() / deg.sum()))

class PNA(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super().__init__()
        
        aggregators = ['mean', 'min', 'max', 'std']
        scalers = ['identity', 'amplification', 'attenuation']
        
        self.conv1 = PNAConv(in_channels, hidden_channels, 
                             aggregators=aggregators, scalers=scalers, 
                             deg=deg,
                             towers=4,
                             divide_input=False)
        self.conv2 = PNAConv(hidden_channels, hidden_channels, aggregators=aggregators, scalers=scalers, deg=deg)
        self.conv3 = PNAConv(hidden_channels, hidden_channels, aggregators=aggregators, scalers=scalers, deg=deg)
        
        self.lin1 = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = F.relu(self.conv3(x, edge_index))
        
        x = global_add_pool(x, batch) 
        
        x = F.relu(self.lin1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        return self.lin2(x)

model_pna = PNA(num_node_features, hidden_channels)
optimizer_pna = torch.optim.Adam(model_pna.parameters(), lr=0.001)
train_model(model_pna, optimizer_pna, "PNA")


===== train start: PNA =====
Best model saved with Val MAE: 1250.9000
Epoch: 01/15, Train MAE: 2086.0595, Val MAE: 1250.9000 (Best: 1250.9000)
Best model saved with Val MAE: 141.1460
Epoch: 02/15, Train MAE: 1949.8272, Val MAE: 141.1460 (Best: 141.1460)
Epoch: 03/15, Train MAE: 1931.2274, Val MAE: 156.9632 (Best: 141.1460)
Epoch: 04/15, Train MAE: 1925.0297, Val MAE: 765.9513 (Best: 141.1460)
Best model saved with Val MAE: 104.0245
Epoch: 05/15, Train MAE: 1926.4879, Val MAE: 104.0245 (Best: 104.0245)
Epoch: 06/15, Train MAE: 1921.8270, Val MAE: 933.1798 (Best: 104.0245)
Epoch: 07/15, Train MAE: 1900.7774, Val MAE: 743.4008 (Best: 104.0245)
Epoch: 08/15, Train MAE: 1916.5491, Val MAE: 315.2469 (Best: 104.0245)
Epoch: 09/15, Train MAE: 1907.4388, Val MAE: 168.8342 (Best: 104.0245)
Epoch: 10/15, Train MAE: 1906.4600, Val MAE: 456.7598 (Best: 104.0245)
Epoch: 11/15, Train MAE: 1900.6672, Val MAE: 770.0054 (Best: 104.0245)
Epoch: 12/15, Train MAE: 1903.6220, Val MAE: 874.6266 (Best: 104.0

/tmp/ipykernel_17915/2230799113.py:66: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(checkpoint_path))


final results PNA MAE: 103.3628


103.36280147980492

In [8]:
from torch_geometric.nn import GATv2Conv, global_add_pool

class GATv2(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, heads=4):
        super().__init__()
        
        self.conv1 = GATv2Conv(in_channels, hidden_channels, heads=heads) 
        self.conv2 = GATv2Conv(hidden_channels * heads, hidden_channels, heads=heads)
        self.conv3 = GATv2Conv(hidden_channels * heads, hidden_channels, heads=1) 

        self.lin1 = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        x = F.elu(self.conv1(x, edge_index))
        x = F.elu(self.conv2(x, edge_index))
        x = F.elu(self.conv3(x, edge_index))
        
        x = global_add_pool(x, batch) 
        
        x = F.relu(self.lin1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        return self.lin2(x)

model_gatv2 = GATv2(num_node_features, hidden_channels, heads=4)
optimizer_gatv2 = torch.optim.Adam(model_gatv2.parameters(), lr=0.001)
train_model(model_gatv2, optimizer_gatv2, "GATv2")


===== train start: GATv2 =====
Best model saved with Val MAE: 602.0786
Epoch: 01/15, Train MAE: 2092.1044, Val MAE: 602.0786 (Best: 602.0786)
Epoch: 02/15, Train MAE: 1784.7776, Val MAE: 745.3424 (Best: 602.0786)
Best model saved with Val MAE: 252.0302
Epoch: 03/15, Train MAE: 1781.5928, Val MAE: 252.0302 (Best: 252.0302)
Epoch: 04/15, Train MAE: 1780.2251, Val MAE: 786.3262 (Best: 252.0302)
Epoch: 05/15, Train MAE: 1787.3259, Val MAE: 502.5818 (Best: 252.0302)
Epoch: 06/15, Train MAE: 1781.9592, Val MAE: 327.7183 (Best: 252.0302)
Epoch: 07/15, Train MAE: 1766.1234, Val MAE: 771.0888 (Best: 252.0302)
Best model saved with Val MAE: 237.1533
Epoch: 08/15, Train MAE: 1769.9386, Val MAE: 237.1533 (Best: 237.1533)
Epoch: 09/15, Train MAE: 1776.4262, Val MAE: 390.6685 (Best: 237.1533)
Epoch: 10/15, Train MAE: 1776.7240, Val MAE: 639.0611 (Best: 237.1533)
Epoch: 11/15, Train MAE: 1775.0261, Val MAE: 484.4454 (Best: 237.1533)
Epoch: 12/15, Train MAE: 1766.7540, Val MAE: 991.3251 (Best: 237.15

/tmp/ipykernel_17915/2230799113.py:66: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(checkpoint_path))


final results GATv2 MAE: 237.0748


237.07477998514855

In [9]:
import torch

def count_parameters(model):
    """
    PyTorchモデルの訓練可能なパラメータの総数を計算する関数
    """
    # model.parameters() は訓練可能な全パラメータ (重みとバイアス) のイテレータを返す
    # p.numel() はテンソル p が持つ要素の総数を返す
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_params

# 以前のステップで定義した GNN モデルのインスタンスを想定
# (model_gin, model_pna, model_gatv2 がメモリにロードされている必要があります)

print(f"==========================================")

# GIN モデルのパラメータ数を計算
if 'model_gin' in locals():
    params_gin = count_parameters(model_gin)
    print(f"GIN モデルの総パラメータ数: {params_gin}")
else:
    print("GIN モデルのインスタンスが見つかりません。")

# PNA モデルのパラメータ数を計算
if 'model_pna' in locals():
    params_pna = count_parameters(model_pna)
    print(f"PNA モデルの総パラメータ数: {params_pna}")
else:
    print("PNA モデルのインスタンスが見つかりません。")

# GATv2 モデルのパラメータ数を計算
if 'model_gatv2' in locals():
    params_gatv2 = count_parameters(model_gatv2)
    print(f"GATv2 モデルの総パラメータ数: {params_gatv2}")
else:
    print("GATv2 モデルのインスタンスが見つかりません。")
    
print(f"==========================================")

GIN モデルの総パラメータ数: 13313
PNA モデルの総パラメータ数: 150069
GATv2 モデルの総パラメータ数: 176001
